In [11]:
import pandas as pd
import datetime

# 1) 파일·시트 목록 불러오기
INPUT_FILE    = r"C:\Users\hanta\Downloads\test.xlsx"
MAPPING_FILE  = r"C:\Users\hanta\Downloads\mapping.xlsx"       # 사용자께서 채워넣은 RawText⇔Category 매핑표
OUTPUT_UNIQUE = r"C:\Users\hanta\Downloads\unique_texts.xlsx"
OUTPUT_SUMM   = r"C:\Users\hanta\Downloads\monthly_summary.xlsx"

# pandas로 워크북의 모든 시트를 dict으로 읽되, 데이터는 4행(0-based index 0~3) 건너뛰고 바로 읽기
sheets: dict[str, pd.DataFrame] = pd.read_excel(
    INPUT_FILE,
    sheet_name=None,
    skiprows=4,
    names=['거래일시','적요','보낸분/받는분','송금메모','출금액','입금액','잔액','거래점','구분']
)

# 2) 고유 텍스트(RawText) 자동 추출
all_texts: list[str] = []
for period, df in sheets.items():
    # 빈 행(거래일시가 없는 행) 제거
    df = df.dropna(subset=['거래일시'])
    # 문자열화 후 리스트에 추가
    all_texts += df['보낸분/받는분'].fillna('').astype(str).tolist()
    all_texts += df['송금메모'].fillna('').astype(str).tolist()

# 중복·빈값 제거
unique_texts = pd.unique([t.strip() for t in all_texts if str(t).strip()])
pd.DataFrame(unique_texts, columns=['RawText']) \
  .to_excel(OUTPUT_UNIQUE, index=False)

print(f"✅ 고유 텍스트 {len(unique_texts)}개를 '{OUTPUT_UNIQUE}'에 저장했습니다.")

# 3) 매핑표 불러오기
#    mapping.xlsx 에는 컬럼명이 RawText, Category 여야 합니다.
mapping_df = pd.read_excel(MAPPING_FILE, dtype=str)
mapping_dict = dict(zip(mapping_df['RawText'], mapping_df['Category']))

# 카테고리 결정 함수
def categorize(text: str) -> str:
    for raw, cat in mapping_dict.items():
        if raw in text:
            return cat
    return '기타'

# 4) 시트별 집계 계산
records = []
for period, df in sheets.items():
    df = df.dropna(subset=['거래일시']).copy()
    # 거래일시를 datetime으로
    df['거래일시'] = pd.to_datetime(df['거래일시'], format='%Y.%m.%d %H:%M:%S', errors='coerce')
    # 키워드 매핑
    df['Category'] = (df['보낸분/받는분'].fillna('') + ' ' + df['송금메모'].fillna('')) \
                      .astype(str).apply(categorize)
    # 금액 합계
    total_in  = df['입금액'].sum()
    total_out = df['출금액'].sum()
    net       = total_in - total_out

    records.append({
        'Period':       period,
        '총입금액':      total_in,
        '총출금액':      total_out,
        '순수입(입금-출금)': net
    })

summary_df = pd.DataFrame(records) \
               .set_index('Period') \
               .sort_index()

# 5) 결과 저장
summary_df.to_excel(OUTPUT_SUMM)
print(f"✅ 월별 요약을 '{OUTPUT_SUMM}'에 저장했습니다.")

# (선택) 카테고리별 집계도 원하면:
# cat_breakdown = df.groupby(['Period','Category'])[['입금액','출금액']].sum().unstack(fill_value=0)
# cat_breakdown.to_excel('category_breakdown.xlsx')


✅ 고유 텍스트 24개를 'C:\Users\hanta\Downloads\unique_texts.xlsx'에 저장했습니다.


C:\Users\hanta\AppData\Local\Temp\ipykernel_32036\579890114.py:28: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_texts = pd.unique([t.strip() for t in all_texts if str(t).strip()])


UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('float64'), dtype('<U1')) -> None

In [12]:
import os
import sys
import pandas as pd

# ───────────────────────────────────────────
# 설정
INPUT_FILE    = r"C:\Users\hanta\Downloads\test.xlsx"          # 원본 엑셀 (시트명이 Period)
MAPPING_FILE  = r"C:\Users\hanta\Downloads\mapping.xlsx"       # 사용자께서 채워넣은 RawText⇔Category 매핑표
OUTPUT_UNIQUE = r"C:\Users\hanta\Downloads\unique_texts.xlsx"  # ② 수동으로 완성할 매핑표
OUTPUT_SUMM   = r"C:\Users\hanta\Downloads\monthly_summary.xlsx" 
# ───────────────────────────────────────────

# 1) 모든 시트 로드
sheets = pd.read_excel(
    INPUT_FILE,
    sheet_name=None,
    skiprows=4,
    names=[
        '거래일시','적요','보낸분/받는분','송금메모',
        '출금액','입금액','잔액','거래점','구분'
    ]
)

# 2) 매핑 파일이 없으면 고유 텍스트만 뽑아서 저장하고 종료
if not os.path.exists(MAPPING_FILE):
    all_texts = []
    for df in sheets.values():
        df = df.dropna(subset=['거래일시'])
        # 문자열 리스트로 변환
        all_texts += df['보낸분/받는분'].fillna('').astype(str).tolist()
        all_texts += df['송금메모'].fillna('').astype(str).tolist()

    unique_texts = pd.unique([t.strip() for t in all_texts if t.strip()])
    pd.DataFrame(unique_texts, columns=['RawText']) \
      .to_excel(OUTPUT_UNIQUE, index=False)

    print(f"✅ '{OUTPUT_UNIQUE}'에 {len(unique_texts)}개를 저장했습니다.")
    print("  → 매핑을 채운 mapping.xlsx를 만든 뒤, 다시 실행하세요.")
    sys.exit(0)

# 3) mapping.xlsx 불러오기
mapping_df   = pd.read_excel(MAPPING_FILE, dtype=str)
mapping_dict = dict(zip(mapping_df['RawText'], mapping_df['Category']))

def categorize(text: str) -> str:
    for raw, cat in mapping_dict.items():
        if raw in text:
            return cat
    return '기타'

# 4) 시트별 집계
records = []
for period, df in sheets.items():
    df = df.dropna(subset=['거래일시']).copy()
    df['거래일시'] = pd.to_datetime(df['거래일시'], errors='coerce')

    # ★ 여기서 각 컬럼을 미리 문자열로 변환해야 에러가 안 납니다 ★
    senders = df['보낸분/받는분'].fillna('').astype(str)
    memos   = df['송금메모'].fillna('').astype(str)
    combined = senders + ' ' + memos      # 이제 둘 다 object dtype → 문자열 덧셈 가능

    df['Category'] = combined.apply(categorize)

    total_in  = pd.to_numeric(df['입금액'], errors='coerce').sum()
    total_out = pd.to_numeric(df['출금액'], errors='coerce').sum()
    net       = total_in - total_out

    records.append({
        'Period':            period,
        '총입금액':           total_in,
        '총출금액':           total_out,
        '순수입(입금-출금)':   net
    })

summary_df = pd.DataFrame(records) \
               .set_index('Period') \
               .sort_index()

summary_df.to_excel(OUTPUT_SUMM)
print(f"✅ 월별 요약을 '{OUTPUT_SUMM}'에 저장했습니다.")

✅ 월별 요약을 'C:\Users\hanta\Downloads\monthly_summary.xlsx'에 저장했습니다.


In [3]:
import os
import sys
import pandas as pd
import datetime

# ───────────────────────────────────────────
# 설정
INPUT_FILE    = r"C:\Users\hanta\Downloads\test.xlsx"
MAPPING_FILE  = r"C:\Users\hanta\Downloads\mapping.xlsx"
OUTPUT_UNIQUE = r"C:\Users\hanta\Downloads\unique_texts.xlsx"
OUTPUT_SUMM   = r"C:\Users\hanta\Downloads\monthly_summary.xlsx"
# ───────────────────────────────────────────

# 1) 모든 시트 로드
sheets = pd.read_excel(
    INPUT_FILE,
    sheet_name=None,
    skiprows=4,
    names=[
        '거래일시','적요','보낸분/받는분','송금메모',
        '출금액','입금액','잔액','거래점','구분'
    ]
)

# 2) 매핑 파일이 없으면 고유 텍스트만 뽑아서 저장하고 종료
# if not os.path.exists(MAPPING_FILE):
#     all_texts = []
#     for df in sheets.values():
#         df = df.dropna(subset=['거래일시'])
#         all_texts += df['보낸분/받는분'].fillna('').astype(str).tolist()
#         all_texts += df['송금메모'].fillna('').astype(str).tolist()
    
#     unique_texts = pd.unique([t.strip() for t in all_texts if t.strip()])
#     pd.DataFrame(unique_texts, columns=['RawText']) \
#       .to_excel(OUTPUT_UNIQUE, index=False)

#     print(f"✅ '{OUTPUT_UNIQUE}'에 {len(unique_texts)}개를 저장했습니다.")
#     print("  → 매핑을 채운 mapping.xlsx를 만든 뒤, 다시 실행하세요.")
#     sys.exit(0)
# 2) mapping.xlsx를 항상 불러와서 매핑 딕셔너리 생성
mapping_df   = pd.read_excel(MAPPING_FILE, dtype=str).fillna('')
mapping_dict = dict(zip(mapping_df['RawText'], mapping_df['Category']))

# 3) mapping.xlsx 불러오기
# Default Category_____________________________________________
# mapping_df   = pd.read_excel(MAPPING_FILE, dtype=str)
# mapping_dict = dict(zip(mapping_df['RawText'], mapping_df['Category']))
# Rev.01 Category_____________________________________________
mapping_df     = pd.read_excel(MAPPING_FILE, dtype=str).fillna('')
mapping_records = mapping_df.to_dict('records')

# # Default Category_____________________________________________
# def categorize(text: str) -> str:
#     for raw, cat in mapping_dict.items():
#         if raw and raw in text:
#             return cat
#     return '기타'
# Rev.01 Category_____________________________________________
def map_fields(text: str):
    for rec in mapping_records:
        raw = rec['RawText']
        if raw and raw in text:
            # rec['입출금']이 '', '입금', '출금' 중 하나
            return rec['Category'], rec.get('입출금','')
    return '기타',''  


# 4) 시트별 집계
records = []
for period, df in sheets.items():
    df = df.dropna(subset=['거래일시']).copy()
    df['거래일시'] = pd.to_datetime(df['거래일시'], errors='coerce')

    # 문자열 결합
    senders  = df['보낸분/받는분'].fillna('').astype(str)
    memos    = df['송금메모'].fillna('').astype(str)
    combined = senders + ' ' + memos

    # Base____________________________________________________________________
    # 금액 숫자형 변환
    df['입금액'] = pd.to_numeric(df['입금액'], errors='coerce').fillna(0)
    df['출금액'] = pd.to_numeric(df['출금액'], errors='coerce').fillna(0)
    
    # 전체 합계
    total_in  = df['입금액'].sum()
    total_out = df['출금액'].sum()
    net       = total_in - total_out

    # ▶ record 딕셔너리 생성 및 병합
    record = {
        'Period':            period,
        '총입금액':           total_in,
        '총출금액':           total_out,
        '순수입(입금-출금)':   net
    }
    
    # # Default Category Agg.____________________________________________________________________
    # # Category 컬럼
    # df['Category'] = combined.apply(categorize)

    # # ▶ Category 기준 합계 계산
    # cat_sums = df.groupby('Category').agg({
    #     '입금액': 'sum',
    #     '출금액': 'sum'
    # })

    # for cat, row in cat_sums.iterrows():
    #     record[f'입금_{cat}']  = row['입금액']
    #     record[f'출금_{cat}']  = row['출금액']
    # records.append(record)

    
    ## Rev.01 Category Agg.____________________________________________________________________    
    df[['Category','InOut']] = combined \
        .apply(lambda x: pd.Series(map_fields(x), index=['Category','InOut']))

    # 3) map_in / map_out 컬럼: InOut 값에 따라 집계 대상 금액 결정
    df['map_in']  = df.apply(lambda r: r['입금액'] if r['InOut'] in ['', '입금'] else 0, axis=1)
    df['map_out'] = df.apply(lambda r: r['출금액'] if r['InOut'] in ['', '출금'] else 0, axis=1)

    # 5) Category 기준 map_in/map_out 집계
    cat_sums = df.groupby('Category').agg({
        'map_in':'sum',
        'map_out':'sum'
    })
    for cat, row in cat_sums.iterrows():
        record[f'입금_{cat}']  = row['map_in']
        record[f'출금_{cat}'] = row['map_out']
    records.append(record)


##____________________________________________________________________    
# 5) 최종 DataFrame 생성 및 저장
summary_df = pd.DataFrame(records) \
               .set_index('Period') \
               .sort_index()
OUTPUT_SUMM = r"C:\Users\hanta\Downloads\monthly_summary_v01.xlsx"
summary_df.to_excel(OUTPUT_SUMM)
print(f"✅ 월별 요약을 '{OUTPUT_SUMM}'에 저장했습니다.")
print(datetime.datetime.now())

ValueError: Columns must be same length as key

In [15]:
import os
import sys
import pandas as pd
import datetime

# ───────────────────────────────────────────
# 설정
INPUT_FILE    = r"C:\Users\hanta\Downloads\test.xlsx"
MAPPING_FILE  = r"C:\Users\hanta\Downloads\mapping.xlsx"
OUTPUT_UNIQUE = r"C:\Users\hanta\Downloads\unique_texts.xlsx"
OUTPUT_SUMM   = r"C:\Users\hanta\Downloads\monthly_summary.xlsx"
# ───────────────────────────────────────────

# 1) 모든 시트 로드
sheets = pd.read_excel(
    INPUT_FILE,
    sheet_name=None,
    skiprows=4,
    names=[
        '거래일시','적요','보낸분/받는분','송금메모',
        '출금액','입금액','잔액','거래점','구분'
    ]
)

# Rev.01 Category_____________________________________________
mapping_df     = pd.read_excel(MAPPING_FILE, dtype=str).fillna('')
mapping_records = mapping_df.to_dict('records')

# Rev.01 Category_____________________________________________
def map_fields(text: str):
    for rec in mapping_records:
        raw = rec['RawText']
        if raw and raw in text:
            # rec['입출금']이 '', '입금', '출금' 중 하나
            return rec['Category'], rec.get('입출금','')
    return '기타',''  


# 4) 시트별 집계
records = []
for period, df in sheets.items():
    df = df.dropna(subset=['거래일시']).copy()
    df['거래일시'] = pd.to_datetime(df['거래일시'], errors='coerce')

    # 문자열 결합
    senders  = df['보낸분/받는분'].fillna('').astype(str)
    memos    = df['송금메모'].fillna('').astype(str)
    combined = senders + ' ' + memos

    # Base____________________________________________________________________
    # 금액 숫자형 변환
    df['입금액'] = pd.to_numeric(df['입금액'], errors='coerce').fillna(0)
    df['출금액'] = pd.to_numeric(df['출금액'], errors='coerce').fillna(0)
    
    # 전체 합계
    total_in  = df['입금액'].sum()
    total_out = df['출금액'].sum()
    net       = total_in - total_out

    # ▶ record 딕셔너리 생성 및 병합
    record = {
        'Period':            period,
        '총입금액':           total_in,
        '총출금액':           total_out,
        '순수입(입금-출금)':   net
    }
    
    ## Rev.01 Category Agg.____________________________________________________________________    
    # 수정: map_fields 결과(튜플)로 바로 DataFrame 생성 → 두 컬럼에 할당
    df[['Category','InOut']] = pd.DataFrame(
        combined.map(map_fields).tolist(),
        index=df.index,
        columns=['Category','InOut']
    )

    # 3) map_in / map_out 컬럼: InOut 값에 따라 집계 대상 금액 결정
    df['map_in']  = df.apply(lambda r: r['입금액'] if r['InOut'] in ['', '입금'] else 0, axis=1)
    df['map_out'] = df.apply(lambda r: r['출금액'] if r['InOut'] in ['', '출금'] else 0, axis=1)

    # 5) Category 기준 map_in/map_out 집계
    cat_sums = df.groupby('Category').agg({
        'map_in':'sum',
        'map_out':'sum'
    })
    for cat, row in cat_sums.iterrows():
        record[f'입금_{cat}']  = row['map_in']
        record[f'출금_{cat}'] = row['map_out']
    records.append(record)


##____________________________________________________________________    
# 5) 최종 DataFrame 생성 및 저장
summary_df = pd.DataFrame(records) \
               .set_index('Period') \
               .sort_index()
OUTPUT_SUMM = r"C:\Users\hanta\Downloads\monthly_summary_v01.xlsx"
summary_df.to_excel(OUTPUT_SUMM)
print(f"✅ 월별 요약을 '{OUTPUT_SUMM}'에 저장했습니다.")
print(datetime.datetime.now())

✅ 월별 요약을 'C:\Users\hanta\Downloads\monthly_summary_v01.xlsx'에 저장했습니다.
2025-06-27 16:23:59.642055


In [4]:
mapping_df     = pd.read_excel(MAPPING_FILE, dtype=str).fillna('')
mapping_records = mapping_df.to_dict('records')

In [20]:
import os
import pandas as pd
import datetime

# ───────────────────────────────────────────
# 설정
INPUT_FILE   = r"C:\Users\hanta\Downloads\test.xlsx"
MAPPING_FILE = r"C:\Users\hanta\Downloads\mapping.xlsx"
OUTPUT_SUMM  = r"C:\Users\hanta\Downloads\monthly_summary_v03.xlsx"
# ───────────────────────────────────────────

# 1) 모든 시트 로드 (헤더 5번째 행부터 컬럼 지정)
sheets = pd.read_excel(
    INPUT_FILE,
    sheet_name=None,
    skiprows=4,
    names=[
        '거래일시','적요','보낸분/받는분','송금메모',
        '출금액','입금액','잔액','거래점','구분'
    ]
)

# 2) 매핑표 불러오기
#    usecols 로 필요한 컬럼만 읽어서 빠르게 처리
mapping_df = (
    pd.read_excel(
        MAPPING_FILE,
        dtype=str,
        usecols=['RawText','Category','입출금']
    )
    .fillna('')  # 빈값은 "" 로
)

# 2-1) (Category, 입출금) 조합 리스트 추출 (중복 제거)
combos = mapping_df[['Category','입출금']] \
           .drop_duplicates() \
           .values.tolist()

# 2-2) 텍스트 → (Category, InOut) 반환 함수
records_map = mapping_df.to_dict('records')
def map_fields(text: str):
    for rec in records_map:
        if rec['RawText'] and rec['RawText'] in text:
            return rec['Category'], rec['입출금']
    return '기타',''


# 3) 시트별 처리 & 레코드 수집
records = []
for period, df in sheets.items():
    df = df.dropna(subset=['거래일시']).copy()
    df['거래일시'] = pd.to_datetime(df['거래일시'], errors='coerce')

    # 금액 컬럼 숫자형으로
    df['입금액'] = pd.to_numeric(df['입금액'], errors='coerce').fillna(0)
    df['출금액'] = pd.to_numeric(df['출금액'], errors='coerce').fillna(0)

    # 보낸분/받는분 + 송금메모 → Category, InOut 컬럼
    combined = (
        df['보낸분/받는분'].fillna('').astype(str)
        + ' '
        + df['송금메모'].fillna('').astype(str)
    )
    df[['Category','InOut']] = pd.DataFrame(
        combined.map(map_fields).tolist(),
        index=df.index,
        columns=['Category','InOut']
    )

    # 3-1) 전체 합계 (변경 없음)
    total_in   = df['입금액'].sum()
    total_out  = df['출금액'].sum()
    net_income = total_in - total_out

    record = {
        'Period':            period,
        '총입금액':           total_in,
        '총출금액':           total_out,
        '순수입(입금-출금)':   net_income
    }

    # 3-2) mapping.xlsx 조합별 집계 추가
    #     Category별・입출금별로 deposit/withdraw sum
    for cat, io in combos:
        if io == '입금':
            # 입금만
            record[f'{cat}_입금'] = df.loc[
                (df['Category']==cat) & (df['InOut']=='입금'),
                '입금액'
            ].sum()
        elif io == '출금':
            # 출금만
            record[f'{cat}_출금'] = df.loc[
                (df['Category']==cat) & (df['InOut']=='출금'),
                '출금액'
            ].sum()
        else:
            # 빈값인 경우 “전체”로 → 양쪽 모두 집계
            record[f'{cat}_입금'] = df.loc[
                df['Category']==cat,
                '입금액'
            ].sum()
            record[f'{cat}_출금'] = df.loc[
                df['Category']==cat,
                '출금액'
            ].sum()

    records.append(record)

# 4) 결과 DataFrame 생성 및 엑셀 저장
summary_df = (
    pd.DataFrame(records)
      .set_index('Period')
      .sort_index()
)
summary_df.to_excel(OUTPUT_SUMM)

print(f"✅ 월별 요약을 '{OUTPUT_SUMM}'에 저장했습니다. 처리 완료: {datetime.datetime.now()}")


✅ 월별 요약을 'C:\Users\hanta\Downloads\monthly_summary_v03.xlsx'에 저장했습니다. 처리 완료: 2025-06-27 16:59:31.947771


In [10]:
mapping_records

[{'RawText': '우체００２건', 'Category': '보험', '입출금': '출금'},
 {'RawText': '536807**39 ,2507-96회차', 'Category': '적금', '입출금': '출금'},
 {'RawText': '토스 남기원', 'Category': '적금', '입출금': '출금'},
 {'RawText': '남기원케이뱅크적금', 'Category': '적금', '입출금': '출금'},
 {'RawText': '최원지적금대출', 'Category': '적금', '입출금': '출금'},
 {'RawText': '#21837111625801', 'Category': '주식', '입출금': '입금'},
 {'RawText': '#21832211431301', 'Category': '주식', '입출금': ''}]

In [13]:
    # 수정: map_fields 결과(튜플)로 바로 DataFrame 생성 → 두 컬럼에 할당
    df[['Category','InOut']] = pd.DataFrame(
        combined.map(map_fields).tolist(),
        index=df.index,
        columns=['Category','InOut']
    )

In [14]:
df

,거래일시,적요,보낸분/받는분,송금메모,출금액,입금액,잔액,거래점,구분,Category,InOut
